In [5]:
import os
import numpy as np
import pandas as pd

import os
import cv2
from pathlib import Path

from skimage.io import imsave, imread

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

from tensorflow.python.framework import ops
ops.reset_default_graph()


In [6]:
#This csv loads the info that will become the masks which is in run-length encoding
train_df = pd.read_csv('~/Data/Metis/Steel/train.csv')
train_df.shape

(50272, 2)

In [3]:
data_path = "/Users/robjohns/Data/Metis/Steel/train_images/"
train_data_path = os.path.join(data_path)
images = os.listdir(train_data_path)
print(len(images))

12568


In [4]:
def name_and_mask(start_idx):
    #in data set, each images has 4 rows, this grabs all 4 and makes sure image name matches
    
    col = start_idx
    img_names = [str(i).split("_")[0] for i in train_df.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
    
    # This takes the 4 values of tagged pixels for each of the 4 defect tags 
    #for the current image
    labels = train_df.iloc[col:col+4, 1]
    
    #makes an empty mask that is 256x1600 pixels with 4 layers for each pixel
    mask = np.zeros((256, 1600, 4), dtype=np.uint8)

    
    #
    for idx, label in enumerate(labels.values):
        
# 4 times, once for each layer, the mask label is processed
# the output will leave all 0's for the mask we made above if there is no code
# or it will be converted to changing the mask on that layer
        
        if label is not np.nan:
            mask_label = np.zeros(1600*256, dtype=np.uint8)
            label = label.split(" ")

#makes a list out of non-zero labels, alternating between positions and lengths
            
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            
#makes lists of positions and lengths by iterating every other, 
#and forces them to become int          
            
            for pos, le in zip(positions, length):
                mask_label[pos-1:pos+le-1] = 1
            mask[:, :, idx] = mask_label.reshape(256, 1600, order='F')
# the positions called in label are turned to 1 in the mask for this layer
    
    return img_names[0], mask

In [5]:
#Make a full table of 4 masks per image with dims [numpix,vertpix,horpix,masks(4)]

In [6]:
#Build Masks

yname=[]
y=[]
for idx in range(0,128,4):
    pix,masks=name_and_mask(idx)
    indy=int(idx/4)
    y.append(masks.astype(np.float32))
    yname.append(pix)
y=np.stack(y, axis=0)  
y_train=y


y_test=[]
for idx in range(0,128,4):
    pix,masks=name_and_mask(idx)
    indy=int(idx/4)
    y_test.append(masks.astype(np.float32))
y_test=np.stack(y_test, axis=0)

In [32]:
#process image files into numpy arrays 
x=[]
for idx in range(0,128,4):    
    name, mask = name_and_mask(idx)
    abs_path = "/Users/robjohns/Data/Metis/Steel/train_images/"
    filename=abs_path+name
    impath = Path(filename)
    img = cv2.imread(filename)
    x.append(img.astype(np.float32))
x=np.stack(x, axis=0)   
x_train=x
x_train=x_train/255

x_test=[]
for idx in range(0,128,4):    
    name, mask = name_and_mask(idx)
    abs_path = "/Users/robjohns/Data/Metis/Steel/train_images/"
    filename=abs_path+name
    impath = Path(filename)
    img = cv2.imread(filename)
    x_test.append(img.astype(np.float32))
x_test=np.stack(x_test, axis=0)
x_test=x_test/255

In [1]:
x_test.shape

NameError: name 'x_test' is not defined

In [7]:

im_idx_list = []
im_idx_list_onlydf = []
im_idx_list_nodf = []


for col in range(0, len(train_df), 4):
    img_names = [str(i).split("_")[0] for i in train_df.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
        
    
    
    labels = train_df.iloc[col:col+4, 1]
    if labels.isna().all():
        im_idx_list_nodf.append(col)
        im_idx_list.append(col)
        
        
    elif (labels.isna() == [False, True, True, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf.append(col)
        
    elif (labels.isna() == [True, False, True, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf.append(col)
        
    elif (labels.isna() == [True, True, False, True]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf.append(col)
        
    elif (labels.isna() == [True, True, True, False]).all():
        im_idx_list.append(col)
        im_idx_list_onlydf.append(col)
        
    else:
        im_idx_list.append(col)
        im_idx_list_onlydf.append(col)

In [8]:
class TFSCGen(keras.utils.Sequence):
    """Generator class for Tensorflow Speech Competition data
    
    args:
        - gen_path (patlib.Path): a path pointing to a directory containing training examples
            examples should be stored in format `gen_path/[class_name]/[file_name].wav`
        - batch_size (int): size of batches to return
        - shuffle (bool): whether or not data is to be shuffled between batches
    
    """
    
    
    def __init__(self, gen_path=im_idx_list, batch_size = 32, shuffle=True):
        
        
        self.gen_files = im_idx_list
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle:
            random.shuffle(self.gen_files)

    def __len__(self):
        """returns the number of examples"""
        
        return int(np.ceil(len(self.gen_files) / float(self.batch_size)))
    
    def on_epoch_end(self):
        """shuffles data after an epoch runs (but only if self.shuffle is set)"""
        
        if self.shuffle:
            random.shuffle(self.gen_files)
            

    def __getitem__(self, idx):
        """function to return the batch given the batch index
        
        args:
            idx (int): this is the batch index generated by keras
        
        """
        
        
        start_idx = idx*self.batch_size
        batch_rows = self.gen_files[start_idx:start_idx+self.batch_size]
        
        x=[]
        for idx in batch_rows:    
            name, mask = name_and_mask(idx)
            abs_path = "/Users/robjohns/Data/Metis/Steel/train_images/"
            #need to generalize this for tests
            
            filename=abs_path+name
            impath = Path(filename)
            img = cv2.imread(filename)
            x.append(img.astype(np.float32))
        x=np.stack(x, axis=0)   
        x=x/255
        
        
        y=[]
        for idx in batch_rows:
            pix,masks=name_and_mask(idx)
            indy=int(idx/4)
            y.append(masks.astype(np.float32))
            yname.append(pix)
        y=np.stack(y, axis=0)  
        
        
        
        return x,y

NameError: name 'keras' is not defined

In [ ]:
#put TFSCGen() where x,y would be in model call

# example:  conv_model.fit(TFSCGen(), validation_data=TFSCGen(tfsc_val),
               epochs=100,
               callbacks=[
                   keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True), 
                   keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, verbose=True)
               ]) 


In [ ]:
#do test train split here, you need to send x_train, y_train, x_test, and y_test to model

In [ ]:
len(x),len(y),len(yname)

In [ ]:
np.save('imgs_masks.npy', y)
np.save('imgs.npy', x)
np.save('imgs_names.npy', yname)

In [ ]:
train_df.shape

In [ ]:
50272/4

In [11]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [2]:
# unet from https://github.com/jocicmarko/ultrasound-nerve-segmentation

In [24]:
def get_unet():
    inputs = Input((256, 1600, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(48, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(48, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(72, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(72, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(108, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(108, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(162, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(162, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(108, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(108, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(72, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(72, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(48, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(48, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(4, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model


In [25]:
unet_model = get_unet()

unet_model.summary()
#check dimensions match expected output. 

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 1600, 3 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 256, 1600, 32 896         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 256, 1600, 32 9248        conv2d_38[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 128, 800, 32) 0           conv2d_39[0][0]                  
____________________________________________________________________________________________

In [26]:
def train_and_predict():

    #'Loading and preprocessing train data
    
   
    imgs_train, imgs_mask_train = x_train, y_train

    
    
    
    #imgs_train = imgs_train/255
    
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    #imgs_train -= mean
    #imgs_train /= std

   

    
    #Creating and compiling model
    
    
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    
    # Fitting model
    
    model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=2, verbose=1, shuffle=True,
              validation_split=0,
              callbacks=[model_checkpoint])

    
    #Loading and preprocessing test data
    
    imgs_test, imgs_id_test = x_test, y_test
    

    imgs_test # /=  255.
    #imgs_test -= mean
    #imgs_test /= std

    
    # Loading saved weights.
    model.load_weights('weights.h5')

    
    #Predicting masks on test data
    
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    
    #convert masks to a table of run length encoding
    
    

In [27]:
train_and_predict()

Train on 32 samples
Epoch 1/2
32/32 [==============================] - 185s 6s/sample - loss: -0.0088 - dice_coef: 0.0088
Epoch 2/2
32/32 [==============================] - 157s 5s/sample - loss: -0.0098 - dice_coef: 0.0098


OSError: Unable to open file (unable to open file: name = 'weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)